In [ ]:
!pip install torch transformers

In [ ]:
!pip install datasets  rouge-score nltk

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e0d93b173903e579f856b3dc6b92d0a92760f4dbc4bfbfcdc466bcdc69e4485f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"deepanrajs21adr010","key":"abf96c248381f7ac588495940c8f6298"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d pariza/bbc-news-summary

Dataset URL: https://www.kaggle.com/datasets/pariza/bbc-news-summary
License(s): CC0-1.0
 56% 5.00M/8.91M [00:00<00:00, 40.0MB/s]
100% 8.91M/8.91M [00:00<00:00, 63.7MB/s]


In [ ]:
!unzip /content/bbc-news-summary.zip

Streaming output truncated to the last 5000 lines.
  inflating: BBC News Summary/Summaries/sport/363.txt  
  inflating: BBC News Summary/Summaries/sport/364.txt  
  inflating: BBC News Summary/Summaries/sport/365.txt  
  inflating: BBC News Summary/Summaries/sport/366.txt  
  inflating: BBC News Summary/Summaries/sport/367.txt  
  inflating: BBC News Summary/Summaries/sport/368.txt  
  inflating: BBC News Summary/Summaries/sport/369.txt  
  inflating: BBC News Summary/Summaries/sport/370.txt  
  inflating: BBC News Summary/Summaries/sport/371.txt  
  inflating: BBC News Summary/Summaries/sport/372.txt  
  inflating: BBC News Summary/Summaries/sport/373.txt  
  inflating: BBC News Summary/Summaries/sport/374.txt  
  inflating: BBC News Summary/Summaries/sport/375.txt  
  inflating: BBC News Summary/Summaries/sport/376.txt  
  inflating: BBC News Summary/Summaries/sport/377.txt  
  inflating: BBC News Summary/Summaries/sport/378.txt  
  inflating: BBC News Summary/Summaries/sport/379.txt

In [ ]:
import os
import re
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def read_files_from_folder(folder_path):
    """
    Reads text files from subfolders in the specified folder path.

    Args:
        folder_path (str): The path to the main folder containing subfolders for different categories.

    Returns:
        list: A list of tuples, each containing the category, filename, and content of a file.
    """

    data = []
    categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if os.path.exists(category_path):
            print(f"Reading files from category: {category}")
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if os.path.isfile(file_path):
                    try:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                    except UnicodeDecodeError:
                        with open(file_path, 'r', encoding='ISO-8859-1') as file:
                            content = file.read()
                    data.append((category, filename, content))
        else:
            print(f"Category folder not found: {category_path}")

    return data

# Paths to the folders
articles_folder = '/content/bbc news summary/BBC News Summary/News Articles'
summaries_folder = '/content/bbc news summary/BBC News Summary/Summaries'

# Read the data
articles_data = read_files_from_folder(articles_folder)
summaries_data = read_files_from_folder(summaries_folder)

# Check if data is read correctly
print(f"Number of articles read: {len(articles_data)}")
print(f"Number of summaries read: {len(summaries_data)}")

# Create DataFrames
df_articles = pd.DataFrame(articles_data, columns=['Category', 'Filename', 'Article'])
df_summaries = pd.DataFrame(summaries_data, columns=['Category', 'Filename', 'Summary'])

# Merge the DataFrames on 'Category' and 'Filename'
data = pd.merge(df_articles, df_summaries, on=['Category', 'Filename'])
data.head()

Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech
Reading files from category: business
Reading files from category: entertainment
Reading files from category: politics
Reading files from category: sport
Reading files from category: tech
Number of articles read: 2225
Number of summaries read: 2225


,Category,Filename,Article,Summary
0,business,486.txt,India's Reliance family feud heats up\n\nThe o...,"However, Anil's demand that Anand Jain - anoth..."
1,business,249.txt,Bush budget seeks deep cutbacks\n\nPresident B...,"President Bush has presented his 2006 budget, ..."
2,business,191.txt,US regulator to rule on pain drug\n\nUS food a...,The European Medicines Agency also said doctor...
3,business,248.txt,Survey confirms property slowdown\n\nGovernmen...,House prices were 11.8% higher on the year in ...
4,business,141.txt,BMW reveals new models pipeline\n\nBMW is prep...,Typically it takes about three years from when...


In [ ]:
df = data[['Article', 'Summary']].copy() # Create a copy of the relevant columns (Article and Summary)
print(df.shape)
df.drop_duplicates(inplace=True)
df.dropna(axis=0,inplace=True)
print(df.shape)

(2225, 2)
(2127, 2)


In [ ]:
# Load the spaCy English model and set up the stop_words
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words

In [ ]:
data.head(1)

,Category,Filename,Article,Summary
0,business,486.txt,India's Reliance family feud heats up\n\nThe o...,"However, Anil's demand that Anand Jain - anoth..."


In [ ]:
# Setting up the contraction_mapping to get rid of contractions in the text
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
def text_cleaner(text, num):
    """
    Cleans a text by performing various preprocessing steps such as removing unnecessary elements and normalizing the text.

    Args:
        text (str): The input text to be cleaned.
        num (int): A flag indicating whether to remove stopwords (0) or not (1).

    Returns:
        str: The cleaned and processed text.
    """
    # Replace newline characters with spaces
    text = text.replace('\n', ' ').replace('\n\n', ' ')

    # Convert to lowercase
    newString = text.lower()

    # Remove content within parentheses
    newString = re.sub(r'\([^)]*\)', '', newString)

    # Remove double quotes
    newString = re.sub('"', '', newString)

    # Replace contractions with their full forms
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])

    # Remove "'s" from words
    newString = re.sub(r"'s\b", "", newString)

    # Remove non-alphabetic characters
    newString = re.sub("[^a-zA-Z]", " ", newString)

    # Replace multiple occurrences of 'm' with 'mm'
    newString = re.sub('[m]{2,}', 'mm', newString)

    # Tokenize and remove stop words (if num == 0)
    if num == 0:
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens = newString.split()

    # Keep only words with more than one character
    long_words = [i for i in tokens if len(i) > 1]
    return (" ".join(long_words)).strip()

In [ ]:
cleaned_texts = []

# clean articles
for t in df["Article"]:
    cleaned_texts.append(text_cleaner(t, 0)) # 0 to remove the stop words and get rid of unnecessary words

# clean summaries
cleaned_summaries = []

for s in df["Summary"]:
    cleaned_summaries.append(text_cleaner(s, 1)) #1 to retain most of the summarized text

# create new dataframe for the cleaned texts and summaries
dataframe = pd.DataFrame(columns=["text", "summary"])

# put cleaned version of texts and summaries into the dataframe
dataframe["text"] = cleaned_texts
dataframe["summary"] = cleaned_summaries

In [ ]:
dataframe.head()

,text,summary
0,parmalat founder offers apology founder boss p...,the founder and former boss of parmalat has ap...
1,budget deficit reach bn budget deficit set hit...,the budget deficit hit record bn in the months...
2,oil prices reach month low oil prices fallen h...,oil prices have fallen heavily for second day ...
3,bmw cash fuel mini production years new mini l...,less than four years after the new mini was la...
4,prepares hybrid onslaught sales hybrid cars se...,ford which has sold of its first hybrid since ...


In [ ]:
dataframe.replace('', np.nan, inplace=True)
dataframe.dropna(axis=0,inplace=True)

In [ ]:
def text_length_percentile(df, text_column, percentile):
    """
    Calculate the text length at which a given percentile of texts are below that length.

    Args:
    df (pandas.DataFrame): The DataFrame containing the text data.
    text_column (str): The column name containing the text data.
    percentile (float): The percentile to calculate (between 0 and 100).

    Returns:
    int: The text length at the given percentile.
    """
    # Calculate the length of each text
    text_lengths = df[text_column].apply(lambda x: len(x.split()))

    # Calculate the length at the given percentile
    length_at_percentile = np.percentile(text_lengths, percentile)

    return int(length_at_percentile)

percentile = 90

length_at_the_percentile = text_length_percentile(dataframe, 'text', percentile)
print(f"{percentile}% of the texts are below this length: {length_at_the_percentile}")

length_at_the_percentile = text_length_percentile(dataframe, 'summary', percentile)
print(f"{percentile}% of the summaries are below this length: {length_at_the_percentile}")

90% of the texts are below this length: 313
90% of the summaries are below this length: 269


In [ ]:
max_input_length = 315
max_target_length = 270

In [ ]:
cleaned_text = np.array(dataframe["text"])
cleaned_summary = np.array(dataframe["summary"])

# define lists for short text and short summaries
short_text = []
short_summary = []

# clean outliers
for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split()) <= max_target_length and len(cleaned_text[i].split()) <= max_input_length):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

# change the dataframe
final_dataframe = pd.DataFrame({"source_text":short_text, "target_text":short_summary})

In [ ]:
dataframe.shape, final_dataframe.shape

((2127, 2), (1880, 2))

In [ ]:
%%time
def lemmatize_text(text):
    doc = nlp(text)
    # Return lemmatized text
    return ' '.join([token.lemma_ for token in doc])

# Apply lemmatization function to DataFrame column
final_dataframe['source_text'] = final_dataframe['source_text'].apply(lemmatize_text)
final_dataframe['target_text'] = final_dataframe['target_text'].apply(lemmatize_text)
final_dataframe.head()

CPU times: user 2min 7s, sys: 356 ms, total: 2min 8s
Wall time: 2min 18s


,source_text,target_text
0,parmalat founder offer apology founder boss pa...,the founder and former boss of parmalat have a...
1,budget deficit reach bn budget deficit set hit...,the budget deficit hit record bn in the month ...
2,oil price reach month low oil price fall heavi...,oil price have fall heavily for second day clo...
3,bmw cash fuel mini production year new mini la...,less than four year after the new mini be laun...
4,prepare hybrid onslaught sale hybrid car set d...,ford which have sell of its first hybrid since...


In [ ]:
final_dataframe["source_text"] = final_dataframe["source_text"].apply(lambda x: "summarize: " + x)

In [ ]:
final_dataframe.head()

,source_text,target_text
0,summarize: parmalat founder offer apology foun...,the founder and former boss of parmalat have a...
1,summarize: budget deficit reach bn budget defi...,the budget deficit hit record bn in the month ...
2,summarize: oil price reach month low oil price...,oil price have fall heavily for second day clo...
3,summarize: bmw cash fuel mini production year ...,less than four year after the new mini be laun...
4,summarize: prepare hybrid onslaught sale hybri...,ford which have sell of its first hybrid since...


In [ ]:
train_df, test_df = train_test_split(final_dataframe, test_size = 0.25)
train_df.shape, test_df.shape

((1410, 2), (470, 2))

In [ ]:
import torch
from datasets import Dataset
from datasets import load_metric


In [ ]:
model_checkpoint ='facebook/bart-base'

In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0


In [ ]:
# from transformers import AutoTokenizer, BertModel
# import torch

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = BertModel.from_pretrained(model_checkpoint)
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the tokenizer and the model
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["source_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train = Dataset.from_pandas(train_df)
valid = Dataset.from_pandas(test_df)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/1410 [00:00<?, ? examples/s]

Map:   0%|          | 0/470 [00:00<?, ? examples/s]

In [ ]:
# from transformers import  DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
batch_size = 4
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
import gc
gc.collect()

22

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-newsarticles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
metric = load_metric("rouge")

The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
eval_dataset = Dataset.from_pandas(test_df)

In [ ]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

In [ ]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

In [ ]:
metrics = predict_results.metrics

In [ ]:
metrics

In [ ]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [ ]:
test_df['source_text'][:2].values

In [ ]:
predictions[:2]

In [ ]:
trainer.evaluate()

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

def generate_summary(example):

   input_ids = tokenizer.encode(example["source_text"], return_tensors="pt", max_length=512, truncation=True).to(device)

   output = model.generate(input_ids)

   summary = tokenizer.decode(output[0], skip_special_tokens=True)

   return {"summary": summary}

summaries = eval_dataset.map(generate_summary, batched=False)

In [ ]:
for i in range(3):

   print("Article:", eval_dataset[i]["source_text"])

   print("\nReference Summary:", eval_dataset[i]["target_text"])

   print("\nGenerated Summary:", summaries[i]["summary"])

   print("\n")

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

def calculate_rouge(reference_list, generated_list):

   rouge = Rouge()

   scores = rouge.get_scores(generated_list, reference_list)

   rouge_1 = sum(score['rouge-1']['f'] for score in scores) / len(scores)

   rouge_2 = sum(score['rouge-2']['f'] for score in scores) / len(scores)

   rouge_l = sum(score['rouge-l']['f'] for score in scores) / len(scores)

   return rouge_1, rouge_2, rouge_l

# Initialize lists to store reference and generated summaries

reference_summaries = [example["target_text"] for example in eval_dataset]

generated_summaries = [example["summary"] for example in summaries]

# Calculate ROUGE scores

rouge_1, rouge_2, rouge_l = calculate_rouge(reference_summaries, generated_summaries)

print("Average ROUGE-1:", rouge_1)

print("Average ROUGE-2:", rouge_2)

print("Average ROUGE-L:", rouge_l)
